In [ ]:
import yt

In [ ]:
ds = yt.load_sample("MOOSE_sample_data/mps_out.e", step=-1)
nt = ds.parameters['_dt'].size

In [ ]:
for i in range(nt):
    ds = yt.load_sample("MOOSE_sample_data/mps_out.e", step=i, displacements = {'connect1': (1.0, [0.0, 0.0, 0.0]), 'connect2': (1.0, [0.0, 0.0, 0.0])})
    for ax in 'xyz':
        s = yt.SlicePlot(ds, ax, ("all", "hydrostatic_stress"))
        s.annotate_mesh_lines(color='k')
        s.save(f"frames/moose_hydrostatic_{ax}_{i:03d}")